AI Project Two - Tic Tac Toe Game - Mar 9th, 2018

Team Member: Baosheng Feng, Jiahang Chen

Team Name & Team ID: Team No1, ID 1025

n <20 & m < 8

n = 18, m = 8

n = 15, m = 7

n = 12, m = 6

n = 6, m = 4

In [1]:
from heapq import heappush
import copy

################### class & function

Define a node class, which is one spot in the tic tac toe game.

In [2]:
class Node:
    def __init__(self, n, m, position, player):
        
        self.position = position
        self.player = player
        
        #row
        if position%n == 0:
            self.row = int(position/n)
        else:
            self.row = int(position/n) + 1
        
        #col
        self.col = position - (self.row-1)*n
        
        #right
        if position%n != 0:
            self.right = position + 1
        else:
            self.right = None
        
        #down
        if position <= (n-1)*n:
            self.down = position + n
        else:
            self.down = None
        
        #lower right
        if self.down and position%n != 0:
            self.rightdown = self.down + 1
        else:
            self.rightdown = None
        
        #lower left
        if self.down and (position-1)%n != 0:
            self.leftdown = self.down -1
        else:
            self.leftdown = None

Define a game class, which will store all the data of a tic tac toe game

In [3]:
class Game:
    
    def __init__(self, n, m):
        
        #0. n, m, "X", "O"
        self.n = n
        self.m = m
        self.human_player = None
        self.ai_player = None
        
        #1. step count
        self.step = 0
        
        #2. list of node, which is used in the winning function
        self.game_detail = initialize_game(n,m)
        
        #3. list of position, which is used to record the game
        self.game_brief = []
        for i in range(n*n):
            self.game_brief.append(i+1)
        
        #4. list of "X" position, used in the winning function
        self.x_position = []
        
        
        #5. list of "O" position, used in the winning function
        self.o_position = []
        
        
        
    #update the game with each move
    def move(self, player, position):
        
        self.step += 1
        self.game_detail[position-1].player = player
        self.game_brief[position-1] = player
        
        if player == "X":
            self.x_position.append(position)
        else:
            self.o_position.append(position)
        
    
    #undo the last action, used in the minimax function
    def undo(self, player, position):
        
        self.step -= 1
        self.game_detail[position-1].player = None
        self.game_brief[position-1] = position
        
        if player == "X":
            self.x_position.remove(position)
        else:
            self.o_position.remove(position)

Initialize the game_detail list which store the game detail information

In [4]:
def initialize_game(n,m):
    
    game = []
    
    for i in range(n*n):
        game.append(Node(n,m,i+1,None))
    
    return game

To reduce the search space, we will focus our search space surround the existing “X” and "O"

In [5]:
def valid_search_space(game):
    
    search_space = []
    
    if game.step == 0:
        return []
    
    if game.step % 2 == 1:
        last_position = game.x_position[len(game.x_position)-1]
    else:
        last_position = game.o_position[len(game.o_position)-1]
        
    #here, we want to limit the search space around the last spot position
    positions = []
    
    row_last_position = game.game_detail[last_position - 1].row
    col_last_position = game.game_detail[last_position - 1].col
    
    extend = 2
    
    start_row = max(row_last_position - extend , 1)
    end_row = min(row_last_position + extend , n)
    start_col = max(col_last_position - extend, 1)
    end_col = min(col_last_position + extend , n)
    
    for row in range(start_row, end_row + 1):
        for col in range(start_col, end_col + 1):
            position = (row - 1)*n + col
            positions.append(position)
    
    for item in positions:
        if game.game_brief[item-1] !="X" and game.game_brief[item-1] !="O":
            search_space.append(item)
    
    
    return search_space
    

Define a function that check the if the player wins

In [6]:
def winning(game, player):
    
    player_list = []
    
    if player == "X":
        player_list = game.x_position
    else:
        player_list = game.o_position
        
    
    if len(player_list) < game.m:
        return False
    
    #horizontal
    for item in player_list:
        #horizontal
        current = game.game_detail[item-1]
        sum = 1
        
        while current.right in player_list:
            current = game.game_detail[current.right-1]
            sum += 1
        
        if sum == game.m:
            return True
        
        
        #vertical
        current = game.game_detail[item-1]
        sum = 1
        
        while current.down in player_list:
            current = game.game_detail[current.down-1]
            sum +=1
        
        if sum == game.m:
            return True
        
        #right diagonal
        current = game.game_detail[item-1]
        sum = 1
        
        while current.rightdown in player_list:
            current = game.game_detail[current.rightdown-1]
            sum +=1
        
        if sum == game.m:
            return True
        
        #left diagonal
        current = game.game_detail[item-1]
        sum = 1
        
        while current.leftdown in player_list:
            current = game.game_detail[current.leftdown-1]
            sum +=1
        
        if sum == game.m:
            return True
    
    return False

Define a function that print the game

In [7]:
def print_game(game):
    
    game_row = game.n
    game_col = game.n
    
    print("Step " + str(game.step))
    
    for row in range(1, game_row + 1):
        col_str = ""
        for col in range(1, game_col + 1):
            position = (row-1)*n + col
            symbol = game.game_brief[position-1]
            if symbol == "X" or symbol == "O":
                col_str += symbol
            else:
                col_str += " "
            if col != n:
                col_str += "|"
        print(col_str)

        if row != n:
            print("-"*(n*2-1))           

minimax algorithm to find the best move

In [8]:
def minimax(game,player,depth,avail_list):
        
    #if the game is over, return value
    if winning(game, "X"):
        return ["X",500 - depth]
    elif winning(game, "O"):
        return ["O",-500 + depth]
    elif game.step == game.n**2:
        return ["Tie",0]
    
    #limit the depth to 4
    if depth > 4:
        return ["Depth", 0]
    
    #available spots
    #availspots=[]    
    #for item in game.game_brief:
        #if item !="X" and item !="O":
            #availspots.append(item)
    
    #for the first "X", we should put it in the middle of the game
    #if len(availspots) == game.n * game.n:
    if game.step == 0:
        if n%2 == 0:
            row = int(n/2)
            col = int(n/2)
        else:
            row = int(n/2)+1
            col = int(n/2)+1
        
        position = (row-1)*game.n+col
        return [position,0]
    
    
    #if the game is not over
    move = []
    
    for position in avail_list:

        game.move(player, position)
        new_list = copy.deepcopy(avail_list)
        new_list.remove(position)
        
        if(player == "X"):
            result = minimax(game, "O", depth + 1, new_list)[1] 
        else:
            result = minimax(game, "X", depth + 1, new_list)[1]
        
        game.undo(player, position)
        
        move.append([position, result])
    
    bestmove = []
    if(player == "X"):
        bestresult = -1000
        for item in move:
            if item[1] > bestresult:
                bestresult = item[1]
                bestmove = item
    else:
        bestresult = 1000
        for item in move:
            if item[1] < bestresult:
                bestresult = item[1]
                bestmove = item
        
    return bestmove

function that find the best next move

In [9]:
def best_next_move(game, player):
    
    next_move = []
    valid_space = valid_search_space(game)
    next_move = minimax(game, player, 0, valid_space)
    
    return next_move

############## Main program

In [43]:
#settings
n = 3
m = 3

game = Game(n,m)

game.human_player = "X"
game.ai_player = "O"

## add moves to here
game.move("X",4)
game.move("O",1)
game.move("X",5)
game.move("O",2)
game.move("X",8)
game.move("O",3)

##print result
print_game(game)

Step 6
O|O|O
-----
X|X| 
-----
 |X| 


In [44]:
##find the best next move
##[0]: position, [1]:score, 0 is tie, 500 is X wins, -500 is O wins
print(best_next_move(game,"X"))

['O', -500]


In [8]:
#from position to row and col
n = 4

position = 4

if position%n == 0:
    row = int(position/n)
else:
    row = int(position/n) + 1

col = position - (row-1)*n

print("Row:" + str(row))
print("Col:" + str(col))

Row:1
Col:4


In [9]:
#from row and col to position
n = 4

row = 3
col = 3

position = (row-1)*n + col

print("Position:" + str(position))

Position:11
